In [1]:
import open3d as o3d 
import numpy as np


In [4]:
print("->正在加载点云... ") 
pcd = o3d.io.read_point_cloud("test.pcd") 
print(pcd)
print("->正在保存点云")
o3d.io.write_point_cloud("write.pcd", pcd) # 默认false，保存为Binarty；True 保存为ASICC形式 print(pcd)
o3d.io.write_point_cloud("write_ascii.pcd", pcd, write_ascii=True)

->正在加载点云... 
PointCloud with 36980 points.
->正在保存点云


True

## Open3D 点云数据读取
open3D 会通过文件后缀判断，点云文件类型。也可以手动输入文件类型。

支持xyz, xyzn, xyzrbg, pts, ply, pcd
手动输入文件类型可以读取 txt

In [18]:
pcd = o3d.io.read_point_cloud("beam_1.txt", format='xyz')
print(np.array(pcd.points).shape)
print(np.array(pcd.normals).shape)
print(np.array(pcd.colors).shape)
pcd = o3d.io.read_point_cloud("beam_1.txt", format='xyzn')
print(np.array(pcd.points).shape)
print(np.array(pcd.normals).shape)
print(np.array(pcd.colors).shape)
pcd = o3d.io.read_point_cloud("1_conferenceRoom_1.npy", format='xyz')
print(np.array(pcd.points).shape)
print(np.array(pcd.normals).shape)
print(np.array(pcd.colors).shape)

(61528, 3)
(0, 3)
(0, 3)
(61528, 3)
(61528, 3)
(0, 3)
(0, 3)
(0, 3)
(0, 3)


## 点云可视化

In [20]:
print("->正在加载点云... ") 
pcd = o3d.io.read_point_cloud("test.pcd") 
print(pcd)
print("->正在可视化点云") 
o3d.visualization.draw_geometries([pcd])

->正在加载点云... 
PointCloud with 36980 points.
->正在可视化点云


In [25]:
import open3d as o3d 
import numpy as np 
print("->正在加载点云1... ") 
pcd1 = o3d.io.read_point_cloud("test1.pcd") 
print(pcd1) 
print("->正在加载点云2...") 
pcd2 = o3d.io.read_point_cloud("beam_1.txt", format='xyzn') 
print(pcd2) 
print("->正在同时可视化两个点云...") 
o3d.visualization.draw_geometries([pcd1, pcd2]) # type: ignore

->正在加载点云1... 
PointCloud with 801147 points.
->正在加载点云2...
PointCloud with 61528 points.
->正在同时可视化两个点云...


In [24]:
import open3d as o3d 
print("->正在加载点云... ") 
pcd = o3d.io.read_point_cloud("test1.pcd") 
print(pcd) 
# 法线估计 
radius = 0.01 
# 搜索半径 
max_nn = 30 # 邻域内用于估算法线的最大点数 
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius, max_nn)) # 执行法线估计 
# 可视化 
o3d.visualization.draw_geometries([pcd], window_name = "可视化参数设置", width = 600, height = 450, left = 30, top = 30, point_show_normal = True)


->正在加载点云... 
PointCloud with 801147 points.


## KD-Tree 与 Octree
- o3d.geometry.KDTreeFlann(pcd) : 建立KDTree
- search_knn_vector_3d(search_Pt, k): KNN
- search_radius_vector_3d(search_Pt, radius)    :半径R近搜索
- search_hybrid_vector_3d(search_pt, radius, max_nn): 混合邻域搜索，返回半径radius 内不超过 max_nn 个近邻点

### KDTree

In [52]:
import open3d as o3d 
import numpy as np 
print("->正在加载点云... ") 
pcd = o3d.io.read_point_cloud("test.pcd") 
print(pcd, np.asarray(pcd.points).shape) 
# 将点云设置为灰色 
pcd.paint_uniform_color([0.5, 0.5, 0.5])

pcd_tree = o3d.geometry.KDTreeFlann(pcd)
print("Paint the 150001st point red.")
pcd.colors[3500] = [1, 0, 0]
print("Find its 200 nearest neighbors, and paint them blue.")
[k, idx, _] = pcd_tree.search_knn_vector_3d(pcd.points[3500], 2000)
np.asarray(pcd.colors)[idx[1:], :] = [0, 0, 1]

print("Find its neighbors with distance less than 0.2, and paint them green.")
[k, idx, _] = pcd_tree.search_radius_vector_3d(pcd.points[3500], 0.1)
np.asarray(pcd.colors)[idx[1:], :] = [0, 1, 0]

print("Find its 200 nearest neighbors, and paint them blue.")
[k, idx, _] = pcd_tree.search_hybrid_vector_3d(pcd.points[3500], 0.1, 1000) # 混合邻域搜索，返回半径radius 内不超过 max_nn 个近邻点
np.asarray(pcd.colors)[idx[1:], :] = [0, 0, 1]

# o3d.visualization.draw_geometries([pcd])


->正在加载点云... 
PointCloud with 36980 points. (36980, 3)
Paint the 150001st point red.
Find its 200 nearest neighbors, and paint them blue.
Find its neighbors with distance less than 0.2, and paint them green.
Find its 200 nearest neighbors, and paint them blue.


### Octree
暂时跳过

In [53]:
import open3d as o3d 
import numpy as np 
# --------------------------- 加载点云 --------------------------- 
print("->正在加载点云... ") 
pcd = o3d.io.read_point_cloud("test1.pcd") 
print("原始点云：", pcd) 
# ============================================================== # ------------------------- 构建Octree -------------------------- 
print('octree 分割') 
octree = o3d.geometry.Octree(max_depth=4) 
octree.convert_from_point_cloud(pcd, size_expand=0.01) 
print("->正在可视化Octree...") 
o3d.visualization.draw_geometries([octree]) 
# ==============================================================


->正在加载点云... 
原始点云： PointCloud with 801147 points.
octree 分割
->正在可视化Octree...


## 点云滤波 and 下采样

体素下采样

1. 对点云进行体素划分
2. 取体素内的点云质心作为该体素点的位置

下采样参数自动设置思路

1. 空间点密度小，划分空间（grid_size， voxel_size） 就大。 反之就小
2. 根据总点数进一步调整，总空间大，增大划分空间。总空间下，减小划分空间。

In [5]:
import open3d as o3d 
import numpy as np 
print("->正在加载点云... ") 
pcd = o3d.io.read_point_cloud("test1.pcd") 
print(pcd) 
print("->正在可视化原始点云") 
o3d.visualization.draw_geometries([pcd]) 
print("->正在体素下采样...") 
voxel_size = 10
downpcd = pcd.voxel_down_sample(voxel_size) 
print(downpcd) 
print("->正在可视化下采样点云") 
o3d.visualization.draw_geometries([downpcd])

np.asarray(downpcd.points)[:, :1] += 1000
print(np.asarray(downpcd.points).shape)
# o3d.visualization.draw_geometries([pcd, downpcd])

->正在加载点云... 
PointCloud with 801147 points.
->正在可视化原始点云
->正在体素下采样...
PointCloud with 60170 points.
->正在可视化下采样点云
(60170, 3)


### 统计滤波
推测操作是计算周围k个点的平均距离

计算标准差

根据给定的阈值，高于阈值的认为是噪声，删去

In [73]:
import open3d as o3d 
import numpy as np 
print("->正在加载点云... ") 
pcd = o3d.io.read_point_cloud("test1.pcd") 
print("原始点云：", pcd) 
# ------------------------- 统计滤波 --------------------------
print("->正在进行统计滤波...") 
# K邻域点的个数
num_neighbors = 100
 
std_ratio = 1
# 标准差乘数 
# # 执行统计滤波，返回滤波后的点云sor_pcd和对应的索引ind 
sor_pcd, ind = pcd.remove_statistical_outlier(num_neighbors, std_ratio) 
sor_pcd.paint_uniform_color([0, 0, 1]) 
print("统计滤波后的点云：", sor_pcd) 
sor_pcd.paint_uniform_color([0, 0, 1]) 
# 提取噪声点云 
sor_noise_pcd = pcd.select_by_index(ind,invert = True) 
print("噪声点云：", sor_noise_pcd) 
sor_noise_pcd.paint_uniform_color([1, 0, 0]) 
# =========================================================== 
# # 可视化统计滤波后的点云和噪声点云 
o3d.visualization.draw_geometries([sor_pcd, sor_noise_pcd])


->正在加载点云... 
原始点云： PointCloud with 801147 points.
->正在进行统计滤波...
统计滤波后的点云： PointCloud with 698559 points.
噪声点云： PointCloud with 102588 points.


In [84]:
import open3d as o3d 
import numpy as np 
print("->正在加载点云... ") 
pcd = o3d.io.read_point_cloud("test1.pcd") 
print("原始点云：", pcd) 
# ------------------------- 半径滤波 -------------------------- 
print("->正在进行半径滤波...") 
num_points = 5
# 邻域球内的最少点数，低于该值的点为噪声点 
radius = 10
# 邻域半径大小 
# # 执行半径滤波，返回滤波后的点云sor_pcd和对应的索引ind 
sor_pcd, ind = pcd.remove_radius_outlier(num_points, radius) 
sor_pcd.paint_uniform_color([0, 0, 1]) 
print("半径滤波后的点云：", sor_pcd) 
sor_pcd.paint_uniform_color([0, 0, 1]) 
# 提取噪声点云 
sor_noise_pcd = pcd.select_by_index(ind,invert = True) 
print("噪声点云：", sor_noise_pcd)
sor_noise_pcd.paint_uniform_color([1, 0, 0])    # 噪点设为红色 
# =========================================================== 
# # 可视化半径滤波后的点云和噪声点云 
o3d.visualization.draw_geometries([sor_pcd, sor_noise_pcd])


->正在加载点云... 
原始点云： PointCloud with 801147 points.
->正在进行半径滤波...
半径滤波后的点云： PointCloud with 797122 points.
噪声点云： PointCloud with 4025 points.
